In [ ]:
library(ape)
library(phytools)
library(RRphylo)
library(dplyr)

In [ ]:
figures_dir = '../fig'

## Load and pre-process data

### Data paths

In [ ]:
euk_tree_nwk = "../data/euk_species_tree.nwk"
snakemake_result_dir = "RESULT_deposit"
gene_structure_stats_tsv = file.path(snakemake_result_dir, 'all_species/intron_lengths.stats')
intron_ratio_distance_tree_nwk = file.path(snakemake_result_dir, 'all_species/intron_ratios_KS_dist.nwk')
intron_length_distance_tree_nwk = file.path(snakemake_result_dir, 'all_species/intron_lengths_KS_dist.nwk')
intron_count_distance_tree_nwk = file.path(snakemake_result_dir, 'all_species/intron_counts_KS_dist.nwk')

### Eukaryotes phylogeny

In [ ]:
euk_tree = read.tree(euk_tree_nwk)
euk_tree

In [ ]:
# replace branches with length < 1 Myr with polytomies
euk_tree = di2multi(euk_tree, tol=1)

In [ ]:
write.tree(euk_tree, file = euk_tree_nwk)

In [ ]:
# Remove terminal branches shorter than 1M years
THRESHOLD = 1

tip.label.to.node.num = function(tree,label){
    return(which(tree$tip.label == label))
}

get_tips = function(node){
    tips = names(tip_parents[which(tip_parents == node)])
}

get.branch.len = function(tree, node){
    return(tree$edge.length[which(tree$edge[,2]==node)])
}

terminal_branch_len = c(-Inf)
while (any(terminal_branch_len < THRESHOLD)){
    # find the node numbers at the tips
    tip_node_num = sapply(euk_tree$tip.label, tip.label.to.node.num, tree=euk_tree)
    # find parent nodes of tips
    tip_parents = sapply(tip_node_num, getParent, tree=euk_tree)
    # get the children for each tip parent
    tip_parents = sapply(as.character(unique(tip_parents)), get_tips)
    # get terminal branch lengths
    #terminal_branch_len = sapply(names(tip_parents), get.branch.len, tree=euk_tree)
    terminal_branch_len = setNames(euk_tree$edge.length[sapply(tip_node_num,
    function(x,y) which(y==x),y=euk_tree$edge[,2])],names(tip_node_num))
    to_drop = c()
    for (parent in names(tip_parents)){
        #if (terminal_branch_len[[parent]] < THRESHOLD){
        if (terminal_branch_len[tip_parents[[parent]][1]] < THRESHOLD){
            to_drop = c(to_drop, tip_parents[[parent]][1])
        }
        else if (length(tip_parents[[parent]]) == 2 & terminal_branch_len[tip_parents[[parent]][2]] < THRESHOLD){
            to_drop = c(to_drop, tip_parents[[parent]][2])
        }
    }
    euk_tree = ape::drop.tip(euk_tree, to_drop)
}

In [ ]:
euk_tree

### Gene structure stats

In [ ]:
gene_structure_stats_df = read.table(gene_structure_stats_tsv, sep='\t', header = TRUE)
gene_structure_stats_df = gene_structure_stats_df[gene_structure_stats_df$Dataset == 'all',]
rownames(gene_structure_stats_df) = gene_structure_stats_df[,1]
# log10 transformation of the mean intron lengths
gene_structure_stats_df$log_Mean = log10(gene_structure_stats_df$Mean)

In [ ]:
gene_structure_stats_df$Mean_intron_ratio_log = log10(gene_structure_stats_df$Mean_intron_ratio)

In [ ]:
# simplify to binomial species names (genus_species)
spec_list = strsplit(rownames(gene_structure_stats_df), '_')
simplify = function(v){
    return(paste(v[1], v[2], sep='_'))
}

gene_structure_stats_df$species = unlist(lapply(spec_list, simplify))

In [ ]:
gene_structure_stats_df = gene_structure_stats_df[gene_structure_stats_df$Mean_intron_ratio > 0,]

In [ ]:
tree_species = euk_tree$tip.label
data_species = gene_structure_stats_df$species

In [ ]:
# remove species not in the species tree
gene_structure_stats_df = gene_structure_stats_df[gene_structure_stats_df$species %in% euk_tree$tip.label,]
# remove tree tips missing from stats table
euk_tree = drop.tip(euk_tree, setdiff(tree_species, data_species))

In [ ]:
# make sure tree and table species all match (should return empty)
setdiff(tree_species, data_species)
setdiff(data_species, tree_species)

### Remove phyla with <10 species
from stats and from tree

In [ ]:
# Find which phyla have at least 10 species
species_by_phylum = as.data.frame(table(gene_structure_stats_df$phylum))
species_by_phylum[species_by_phylum$Freq >= 10,]

In [ ]:
# list of phyla with >=10 species and corresponding list of species
phyla10 = species_by_phylum[species_by_phylum$Freq >= 10,'Var1']
phyla10_species = gene_structure_stats_df[gene_structure_stats_df$phylum %in% phyla10,'species']

In [ ]:
# create tree containing only species from phyla with >=10 species
euk_tree_phyla10 = drop.tip(euk_tree,euk_tree$tip.label[-match(phyla10_species, euk_tree$tip.label)])

In [ ]:
euk_tree_phyla10

In [ ]:
write.tree(euk_tree_phyla10, "../data/euk_species_tree_phyla10.nwk")

In [ ]:
# check if the remaining phyla are monophyletic
for (phyl in phyla10){
    phyl_species = gene_structure_stats_df[gene_structure_stats_df$phylum == phyl,]$species
    phyl_mono = is.monophyletic(euk_tree_phyla10, phyl_species)
    print(paste0(phyl, ' is monophyletic: ', phyl_mono))
}

In [ ]:
gene_structure_stats_phyla10_df = gene_structure_stats_df[gene_structure_stats_df$species %in% phyla10_species,]

## Test for differences among phyla
Analyze several key gene structure features to determine the differences between phyla. The analysis steps are:
1. Run RRphylo to determine per-branch rates
2. Rescale the phylogeny according to inferred rates, thereby accounting for rate heterogenity and avoiding an oversimplifies BM model
3. Perform phylogenetic ANOVA test

In [ ]:
species_phylum = gene_structure_stats_df$phylum
names(species_phylum) = gene_structure_stats_df$species
phyla10_species_phylum = gene_structure_stats_phyla10_df$phylum
names(phyla10_species_phylum) = gene_structure_stats_phyla10_df$species

In [ ]:
analyze_feature = function(feature, transform=NULL){
    species_data = gene_structure_stats_df[[feature]]
    if (! is.null(transform)){
        species_data = sapply(species_data, transform)
    }
    names(species_data) = gene_structure_stats_df$species
    rrphylo = RRphylo(tree = euk_tree, y = species_data)
    rescaled_tree = rescaleRR(euk_tree, RR=rrphylo)
    # subset tree and data for phyla with >10 species
    rescaled_tree_phyla10 = drop.tip(rescaled_tree,rescaled_tree$tip.label[-match(phyla10_species, rescaled_tree$tip.label)])
    species_data_phyla10 = species_data[phyla10_species]
    # phylogenetic ANOVA
    set.seed(100)
    phyl_anova = phylANOVA(rescaled_tree_phyla10, phyla10_species_phylum, species_data_phyla10)
    # Add inferred ancestral states as node labels
    rrphylo$tree$node.label = round(rrphylo$aces[,1], digits=2)
    # prepare RRphylo plots
    rrphylo_plot = plotRR(rrphylo, species_data)
    return (list(data=species_data, rrphylo=rrphylo, phyl_ANOVA=phyl_anova, rrphylo_plot=rrphylo_plot))
}

#### Mean intron ratio
log10 transformed

In [ ]:
intron_ratio_res = analyze_feature('Mean_intron_ratio', log10)
intron_ratio_res$phyl_ANOVA$Pf

#### Mean total exon/intron length per transcript
log10 transformed

In [ ]:
tot_exon_length_res = analyze_feature('Mean_total_exon_length_per_transcript', log10)
tot_exon_length_res$phyl_ANOVA$Pf

In [ ]:
tot_intron_length_res = analyze_feature('Mean_total_intron_length_per_transcript', log10)
tot_intron_length_res$phyl_ANOVA$Pf

#### Mean intron length

In [ ]:
mean_intron_length_res = analyze_feature('Mean', log10)
mean_intron_length_res$phyl_ANOVA$Pf

In [ ]:
mean_intron_length_res$phyl_ANOVA$Pt

#### Mean number of introns

In [ ]:
mean_intron_count_res = analyze_feature('Mean_per_transcript')
mean_intron_count_res$phyl_ANOVA$Pf

In [ ]:
mean_intron_count_res$phyl_ANOVA$Pt

## Ancestral states at phyla MRCAs
Find the inferred ancestral states for key gene structure features at the MRCAs of phyla.

In [ ]:
# state and rate at phyla MRCAs
mrca_mean_len = c()
mrca_mean_count = c()
mrca_mean_ratio = c()
for (phyl in phyla10){
    phyl_species = gene_structure_stats_df[gene_structure_stats_df$phylum == phyl,]$species
    phyl_mrca = findMRCA(mean_intron_count_res$rrphylo$tree, phyl_species)
    mrca_count = mean_intron_count_res$rrphylo$aces[as.character(phyl_mrca),]
    mrca_mean_count = c(mrca_mean_count, mrca_count)
    phyl_mrca = findMRCA(mean_intron_length_res$rrphylo$tree, phyl_species)
    mrca_len = mean_intron_length_res$rrphylo$aces[as.character(phyl_mrca),]
    mrca_len = 10**mrca_len
    mrca_mean_len = c(mrca_mean_len, mrca_len)
    phyl_mrca = findMRCA(intron_content_res$rrphylo$tree, phyl_species)
    mrca_ratio = intron_ratio_res$rrphylo$aces[as.character(phyl_mrca),]
    mrca_ratio = 10**mrca_ratio
    mrca_mean_ratio = c(mrca_mean_ratio, mrca_ratio)
}

In [ ]:
mrca_states = cbind(as.character(phyla10), round(mrca_mean_ratio, digits=2), round(mrca_mean_count, digits=2), round(mrca_mean_len, digits=2))
mrca_states = as.data.frame(mrca_states)
colnames(mrca_states) = c('Phylum', 'Mean intron ratio', 'Mean intron count', 'Mean intron length')
mrca_states

What about the eukaryotic root?

In [ ]:
print(10**intron_ratio_res$rrphylo$aces[1])
print(10**mean_intron_length_res$rrphylo$aces[1])
print(mean_intron_count_res$rrphylo$aces[1])

In [ ]:
# Assign ancestral states to node labels and write tree (only including phyla10 - for use in other notebooks)
# intron ratio
intron_ratio_res$rrphylo$tree$node.label = intron_ratio_res$rrphylo$aces
write.tree(keep.tip(intron_ratio_res$rrphylo$tree, phyla10_species), file='../data/euk_species_tree_phyla10_intron_ratio_anc.nwk')

# intron length
mean_intron_length_res$rrphylo$tree$node.label = mean_intron_length_res$rrphylo$aces
write.tree(keep.tip(mean_intron_length_res$rrphylo$tree, phyla10_species), file='../data/euk_species_tree_phyla10_intron_length_anc.nwk')

# intron count
mean_intron_count_res$rrphylo$tree$node.label = mean_intron_count_res$rrphylo$aces
write.tree(keep.tip(mean_intron_count_res$rrphylo$tree, phyla10_species), file='../data/euk_species_tree_phyla10_n_introns_anc.nwk')

#### RRphylo plot
Only phyla with >=10 species

In [ ]:
euk_tree_phyla10 = di2multi(euk_tree_phyla10, tol=5)
euk_tree_phyla10 = drop.tip(euk_tree_phyla10, c('trichuris_muris', 'trichinella_spiralis'))

In [ ]:
colors = c('#1F77B4',
 '#FF7F0E',
 '#2CA02C',
 '#D62728',
 '#9467BD',
 '#8C564B',
 '#E377C2',
 '#7F7F7F',
 '#BCBD22',
 '#17BECF')

In [ ]:
phy_order = c('Chordata', 'Arthropoda', 'Mollusca', 'Cnidaria', 'Nematoda', 'Ascomycota', 'Streptophyta')

__Mean intron ratio__

In [ ]:
species_data = gene_structure_stats_phyla10_df$Mean_intron_ratio_log
names(species_data) = gene_structure_stats_phyla10_df$species
rrphylo = RRphylo(tree = euk_tree_phyla10, y = species_data)

In [ ]:
rrphylo_plot = plotRR(rrphylo, species_data)

In [ ]:
options(repr.plot.width=30, repr.plot.height=60)
par(mar = c(2, 2, 2, 2))
color_pal = colorRampPalette(c("gold", "darkorchid"))
tree_args = list(edge.width=5, no.margin=FALSE)
colorbar_args = list(direction='horizontal',x=2,y=-4, height=0.5,width=1.5,border=NA,title.pos='bottom',labs.cex=2,title='Mean intron ratio (log10)',title.cex=3,title.adj=c(0.5,-5))
rrphylo_plot$plotRRphen(colorbar.args = colorbar_args, color.pal=color_pal, tree.args=tree_args)

# add phylum labels
i = 1
for (phyl in phy_order){
    phyl_color = colors[i]
    phyl_species = gene_structure_stats_phyla10_df[gene_structure_stats_phyla10_df$phylum == phyl,]$species
    phyl_species = intersect(phyl_species, rrphylo$tree$tip.label)
    mrca_node = getMRCA(rrphylo$tree, phyl_species)
    mrca_state = mrca_states[mrca_states$Phylum == phyl, 'Mean intron ratio']
    mrca_state = round(log10(as.numeric(mrca_state)), digits=2)
    nodelabels(node = mrca_node, pch=21, col=phyl_color, bg=phyl_color, cex=6)
    nodelabels(node = mrca_node, text=mrca_state, adj=c(1,-0.55), col=phyl_color, frame='none', cex=4)
    tips<-getDescendants(rrphylo$tree, mrca_node)
    tips<-tips[tips<=Ntip(rrphylo$tree)]
    pp = get("last_plot.phylo", envir = .PlotPhyloEnv)
    y<-range(pp$yy[tips])
    lines(x=c(1650,1650), y=c(y[1],y[2]), lwd=20, col=phyl_color)
    i = i+1
}
# add root label
phyl_species = gene_structure_stats_phyla10_df$species
phyl_species = intersect(phyl_species, rrphylo$tree$tip.label)
root_node = getMRCA(rrphylo$tree, phyl_species)
root_state = round(intron_ratio_res$rrphylo$aces[1],2)
nodelabels(node = root_node, pch=21, col='black', bg='black', cex=6)
nodelabels(node = root_node, text=root_state, adj=c(-0.3,-0.55), col='black', frame='none', cex=4)

In [ ]:
fig_name = 'fig_3a'
fig_path = file.path(figures_dir, paste0(fig_name,'.pdf'))
pdf(fig_path, width=30, height=60)
options(repr.plot.width=30, repr.plot.height=60)
par(mar = c(2, 2, 2, 2))

rrphylo_plot$plotRRphen(colorbar.args = colorbar_args, color.pal=color_pal, tree.args=tree_args)

# add phylum labels
i = 1
for (phyl in phy_order){
    phyl_color = colors[i]
    phyl_species = gene_structure_stats_phyla10_df[gene_structure_stats_phyla10_df$phylum == phyl,]$species
    phyl_species = intersect(phyl_species, rrphylo$tree$tip.label)
    mrca_node = getMRCA(rrphylo$tree, phyl_species)
    mrca_state = mrca_states[mrca_states$Phylum == phyl, 'Mean intron ratio']
    mrca_state = round(log10(as.numeric(mrca_state)), digits=2)
    nodelabels(node = mrca_node, pch=21, col=phyl_color, bg=phyl_color, cex=6)
    nodelabels(node = mrca_node, text=mrca_state, adj=c(1,-0.55), col=phyl_color, frame='none', cex=4)
    tips<-getDescendants(rrphylo$tree, mrca_node)
    tips<-tips[tips<=Ntip(rrphylo$tree)]
    pp = get("last_plot.phylo", envir = .PlotPhyloEnv)
    y<-range(pp$yy[tips])
    lines(x=c(1650,1650), y=c(y[1],y[2]), lwd=20, col=phyl_color)
    i = i+1
}
# add root label
phyl_species = gene_structure_stats_phyla10_df$species
phyl_species = intersect(phyl_species, rrphylo$tree$tip.label)
root_node = getMRCA(rrphylo$tree, phyl_species)
root_state = round(intron_ratio_res$rrphylo$aces[1],2)
nodelabels(node = root_node, pch=21, col='black', bg='black', cex=6)
nodelabels(node = root_node, text=root_state, adj=c(-0.3,-0.55), col='black', frame='none', cex=4)
dev.off()

__Mean intron length__

In [ ]:
species_data = gene_structure_stats_phyla10_df$log_Mean
names(species_data) = gene_structure_stats_phyla10_df$species
rrphylo = RRphylo(tree = euk_tree_phyla10, y = species_data)
rrphylo_plot = plotRR(rrphylo, species_data)

In [ ]:
options(repr.plot.width=30, repr.plot.height=60)
par(mar = c(2, 2, 2, 2))
color_pal = colorRampPalette(c("gold", "darkorchid"))
tree_args = list(edge.width=5, no.margin=FALSE)
colorbar_args = list(direction='horizontal',x=2,y=-4, height=0.5,width=1.5,border=NA,title.pos='bottom',labs.cex=2,title='Mean intron length (log10 bp)',title.cex=3,title.adj=c(0.5,-5))
rrphylo_plot$plotRRphen(colorbar.args = colorbar_args, color.pal=color_pal, tree.args=tree_args)

# add phylum labels
i = 1
for (phyl in phy_order){
    phyl_color = colors[i]
    phyl_species = gene_structure_stats_phyla10_df[gene_structure_stats_phyla10_df$phylum == phyl,]$species
    phyl_species = intersect(phyl_species, rrphylo$tree$tip.label)
    mrca_node = getMRCA(rrphylo$tree, phyl_species)
    mrca_state = mrca_states[mrca_states$Phylum == phyl, 'Mean intron length']
    mrca_state = round(log10(as.numeric(mrca_state)), digits=2)
    nodelabels(node = mrca_node, pch=21, col=phyl_color, bg=phyl_color, cex=6)
    nodelabels(node = mrca_node, text=mrca_state, adj=c(1,-0.55), col=phyl_color, frame='none', cex=4)
    tips<-getDescendants(rrphylo$tree, mrca_node)
    tips<-tips[tips<=Ntip(rrphylo$tree)]
    pp = get("last_plot.phylo", envir = .PlotPhyloEnv)
    y<-range(pp$yy[tips])
    lines(x=c(1650,1650), y=c(y[1],y[2]), lwd=20, col=phyl_color)
    i = i+1
}
# add root label
phyl_species = gene_structure_stats_phyla10_df$species
phyl_species = intersect(phyl_species, rrphylo$tree$tip.label)
root_node = getMRCA(rrphylo$tree, phyl_species)
root_state = round(mean_intron_length_res$rrphylo$aces[1],2)
nodelabels(node = root_node, pch=21, col='black', bg='black', cex=6)
nodelabels(node = root_node, text=root_state, adj=c(-0.3,-0.55), col='black', frame='none', cex=4)

In [ ]:
fig_name = 'fig_S1a'
fig_path = file.path(figures_dir, paste0(fig_name,'.pdf'))
pdf(fig_path, width=30, height=60)
options(repr.plot.width=30, repr.plot.height=60)
par(mar = c(2, 2, 2, 2))

rrphylo_plot$plotRRphen(colorbar.args = colorbar_args, color.pal=color_pal, tree.args=tree_args)

# add phylum labels
i = 1
for (phyl in phy_order){
    phyl_color = colors[i]
    phyl_species = gene_structure_stats_phyla10_df[gene_structure_stats_phyla10_df$phylum == phyl,]$species
    phyl_species = intersect(phyl_species, rrphylo$tree$tip.label)
    mrca_node = getMRCA(rrphylo$tree, phyl_species)
    mrca_state = mrca_states[mrca_states$Phylum == phyl, 'Mean intron length']
    mrca_state = round(log10(as.numeric(mrca_state)), digits=2)
    nodelabels(node = mrca_node, pch=21, col=phyl_color, bg=phyl_color, cex=6)
    nodelabels(node = mrca_node, text=mrca_state, adj=c(1,-0.55), col=phyl_color, frame='none', cex=4)
    tips<-getDescendants(rrphylo$tree, mrca_node)
    tips<-tips[tips<=Ntip(rrphylo$tree)]
    pp = get("last_plot.phylo", envir = .PlotPhyloEnv)
    y<-range(pp$yy[tips])
    lines(x=c(1650,1650), y=c(y[1],y[2]), lwd=20, col=phyl_color)
    i = i+1
}
# add root label
phyl_species = gene_structure_stats_phyla10_df$species
phyl_species = intersect(phyl_species, rrphylo$tree$tip.label)
root_node = getMRCA(rrphylo$tree, phyl_species)
root_state = round(mean_intron_length_res$rrphylo$aces[1],2)
nodelabels(node = root_node, pch=21, col='black', bg='black', cex=6)
nodelabels(node = root_node, text=root_state, adj=c(-0.3,-0.55), col='black', frame='none', cex=4)

dev.off()

__Mean intron count__

In [ ]:
species_data = gene_structure_stats_phyla10_df$Mean_per_transcript
names(species_data) = gene_structure_stats_phyla10_df$species
rrphylo = RRphylo(tree = euk_tree_phyla10, y = species_data)
rrphylo_plot = plotRR(rrphylo, species_data)

In [ ]:
options(repr.plot.width=30, repr.plot.height=60)
par(mar = c(2, 2, 2, 2))
color_pal = colorRampPalette(c("gold", "darkorchid"))
tree_args = list(edge.width=5, no.margin=FALSE)
colorbar_args = list(direction='horizontal',x=2,y=-4, height=0.5,width=1.5,border=NA,title.pos='bottom',labs.cex=2,title='Mean number of introns',title.cex=3,title.adj=c(0.5,-5))
rrphylo_plot$plotRRphen(colorbar.args = colorbar_args, color.pal=color_pal, tree.args=tree_args)

# add phylum labels
i = 1
for (phyl in phy_order){
    phyl_color = colors[i]
    phyl_species = gene_structure_stats_phyla10_df[gene_structure_stats_phyla10_df$phylum == phyl,]$species
    phyl_species = intersect(phyl_species, rrphylo$tree$tip.label)
    mrca_node = getMRCA(rrphylo$tree, phyl_species)
    mrca_state = mrca_states[mrca_states$Phylum == phyl, 'Mean intron count']
    mrca_state = round(as.numeric(mrca_state), digits=2)
    nodelabels(node = mrca_node, pch=21, col=phyl_color, bg=phyl_color, cex=6)
    nodelabels(node = mrca_node, text=mrca_state, adj=c(1,-0.55), col=phyl_color, frame='none', cex=4)
    tips<-getDescendants(rrphylo$tree, mrca_node)
    tips<-tips[tips<=Ntip(rrphylo$tree)]
    pp = get("last_plot.phylo", envir = .PlotPhyloEnv)
    y<-range(pp$yy[tips])
    lines(x=c(1650,1650), y=c(y[1],y[2]), lwd=20, col=phyl_color)
    i = i+1
}
# add root label
phyl_species = gene_structure_stats_phyla10_df$species
phyl_species = intersect(phyl_species, rrphylo$tree$tip.label)
root_node = getMRCA(rrphylo$tree, phyl_species)
root_state = round(mean_intron_count_res$rrphylo$aces[1],2)
nodelabels(node = root_node, pch=21, col='black', bg='black', cex=6)
nodelabels(node = root_node, text=root_state, adj=c(-0.3,-0.55), col='black', frame='none', cex=4)

In [ ]:
fig_name = 'fig_S2a'
fig_path = file.path(figures_dir, paste0(fig_name,'.pdf'))
pdf(fig_path, width=30, height=60)
options(repr.plot.width=30, repr.plot.height=60)
par(mar = c(2, 2, 2, 2))

rrphylo_plot$plotRRphen(colorbar.args = colorbar_args, color.pal=color_pal, tree.args=tree_args)

# add phylum labels
i = 1
for (phyl in phy_order){
    phyl_color = colors[i]
    phyl_species = gene_structure_stats_phyla10_df[gene_structure_stats_phyla10_df$phylum == phyl,]$species
    phyl_species = intersect(phyl_species, rrphylo$tree$tip.label)
    mrca_node = getMRCA(rrphylo$tree, phyl_species)
    mrca_state = mrca_states[mrca_states$Phylum == phyl, 'Mean intron count']
    mrca_state = round(as.numeric(mrca_state), digits=2)
    nodelabels(node = mrca_node, pch=21, col=phyl_color, bg=phyl_color, cex=6)
    nodelabels(node = mrca_node, text=mrca_state, adj=c(1,-0.55), col=phyl_color, frame='none', cex=4)
    tips<-getDescendants(rrphylo$tree, mrca_node)
    tips<-tips[tips<=Ntip(rrphylo$tree)]
    pp = get("last_plot.phylo", envir = .PlotPhyloEnv)
    y<-range(pp$yy[tips])
    lines(x=c(1650,1650), y=c(y[1],y[2]), lwd=20, col=phyl_color)
    i = i+1
}
# add root label
phyl_species = gene_structure_stats_phyla10_df$species
phyl_species = intersect(phyl_species, rrphylo$tree$tip.label)
root_node = getMRCA(rrphylo$tree, phyl_species)
root_state = round(mean_intron_count_res$rrphylo$aces[1],2)
nodelabels(node = root_node, pch=21, col='black', bg='black', cex=6)
nodelabels(node = root_node, text=root_state, adj=c(-0.3,-0.55), col='black', frame='none', cex=4)

dev.off()

## Modeling gene structure
PGLS models of gene structure across species.

In [ ]:
# log-transform total exon and intron length
rownames(gene_structure_stats_df) = gene_structure_stats_df$species
gene_structure_stats_df$Mean_total_exon_length_per_transcript_log = log10(gene_structure_stats_df$Mean_total_exon_length_per_transcript)
gene_structure_stats_df$Mean_total_intron_length_per_transcript_log = log10(gene_structure_stats_df$Mean_total_intron_length_per_transcript)

### intron ratio

In [ ]:
pgls = PGLS_fossil(modform= Mean_intron_ratio_log ~ Mean_total_intron_length_per_transcript_log, data=gene_structure_stats_df, RR=intron_ratio_res$rrphylo)
summary(pgls)

In [ ]:
pgls = PGLS_fossil(modform= Mean_intron_ratio_log ~ Mean_total_exon_length_per_transcript_log, data=gene_structure_stats_df, RR=intron_ratio_res$rrphylo)
summary(pgls)

In [ ]:
pgls = PGLS_fossil(modform= Mean_intron_ratio_log ~ Mean_total_exon_length_per_transcript_log + Mean_total_intron_length_per_transcript_log, data=gene_structure_stats_df, RR=intron_ratio_res$rrphylo)
summary(pgls)

In [ ]:
pgls = PGLS_fossil(modform= Mean_intron_ratio_log ~ Mean_total_exon_length_per_transcript_log * Mean_total_intron_length_per_transcript_log, data=gene_structure_stats_df, RR=intron_ratio_res$rrphylo)
summary(pgls)

### intron length / intron count

In [ ]:
pgls = PGLS_fossil(modform= Mean_total_intron_length_per_transcript_log ~ log_Mean * Mean_per_transcript, data=gene_structure_stats_df, RR=tot_intron_length_res$rrphylo)
summary(pgls)

In [ ]:
pgls = PGLS_fossil(modform= Mean_total_intron_length_per_transcript_log ~ log_Mean + Mean_per_transcript, data=gene_structure_stats_df, RR=tot_intron_length_res$rrphylo)
summary(pgls)

In [ ]:
pgls = PGLS_fossil(modform= Mean_total_intron_length_per_transcript_log ~ log_Mean, data=gene_structure_stats_df, RR=tot_intron_length_res$rrphylo)
summary(pgls)

In [ ]:
pgls = PGLS_fossil(modform= Mean_total_intron_length_per_transcript_log ~ Mean_per_transcript, data=gene_structure_stats_df, RR=tot_intron_length_res$rrphylo)
summary(pgls)

#### Per-phylum

In [ ]:
summarize_pgls = function(pgls){
    pgls_sum = summary(pgls)
    pgls_stats = list()
    pgls_stats$R2 = max(pgls_sum$adj.r.squared, 0)
    pgls_stats$AIC = pgls_sum$aic
    pgls_coef = as.data.frame(pgls_sum$coefficients)
    for (c in rownames(pgls_coef)){
        pgls_stats[[c]] = pgls_coef[c,'Estimate']
        pgls_stats[[paste0(c,'_p')]] = pgls_coef[c,'p.value']
    }
    return(pgls_stats)
}

In [ ]:
models = c(Mean_intron_ratio_log ~ Mean_total_intron_length_per_transcript_log,
           Mean_intron_ratio_log ~ Mean_total_exon_length_per_transcript_log,
           Mean_intron_ratio_log ~ Mean_total_exon_length_per_transcript_log + Mean_total_intron_length_per_transcript_log,
           Mean_total_intron_length_per_transcript_log ~ log_Mean,
           Mean_total_intron_length_per_transcript_log ~ Mean_per_transcript,
           Mean_total_intron_length_per_transcript_log ~ log_Mean + Mean_per_transcript
)
names(models) = paste0('M',seq(1,6))

In [ ]:
res = c()
phyla = c('All', as.vector(phyla10))
for (phyl in phyla){
    print(phyl)
    # extract phylum data and tree
    if (phyl == 'All'){
        phyl_data = gene_structure_stats_df[gene_structure_stats_df$species %in% euk_tree$tip.label,]
    }else{
        phyl_data = gene_structure_stats_df[gene_structure_stats_df$phylum == phyl,]
    }   
    phyl_species = phyl_data$species
    phyl_tree = keep.tip(euk_tree, phyl_species)
    # prepare data for RRphylo
    phyl_intron_ratio = phyl_data$Mean_intron_ratio_log
    names(phyl_intron_ratio) = phyl_species
    phyl_tot_intron_len = phyl_data$Mean_total_intron_length_per_transcript_log
    names(phyl_tot_intron_len) = phyl_species
    # run RRphylo
    phyl_intron_ratio_rr = RRphylo(tree = phyl_tree, y = phyl_intron_ratio)
    phyl_tot_intron_len_rr = RRphylo(tree = phyl_tree, y = phyl_tot_intron_len)
    # run PGLSs
    phyl_model_stats = c()
    for (model in names(models)){
        model_formula = models[[model]]
        dep = all.vars(model_formula)[1]
        if (dep == "Mean_intron_ratio_log"){
            rr = phyl_intron_ratio_rr
        }else if (dep == "Mean_total_intron_length_per_transcript_log"){
            rr = phyl_tot_intron_len_rr
        }
        pgls =  PGLS_fossil(modform=model_formula, data=gene_structure_stats_df, RR=rr)
        pgls_stats = unlist(summarize_pgls(pgls))
        names(pgls_stats) = paste(model, names(pgls_stats), sep = '_')
        phyl_model_stats = c(phyl_model_stats, pgls_stats)
    }
    res = rbind(res, phyl_model_stats)
}

res = as.data.frame(res)
rownames(res) = phyla

In [ ]:
rownames(res) = phyla
res

In [ ]:
write.table(res, sep='\t', file='../data/per_phylum_models.tsv')

## Distance tree plot
Plotting the eukaryotic tree with branch lengths adjusted according to a distance matrix (based on gene structure feature distributions).

__Intron ratio__

In [ ]:
intron_ratio_distance_tree = read.tree(intron_ratio_distance_tree_nwk)

In [ ]:
# simlify full species names to binomial names
spec_list = strsplit(intron_ratio_distance_tree$tip.label, '_')
intron_ratio_distance_tree$tip.label = unlist(lapply(spec_list, simplify))

In [ ]:
# keep only phyla with >=10 sspecies
intron_ratio_distance_phyla10_tree = keep.tip(intron_ratio_distance_tree, phyla10_species)

In [ ]:
# change branches with negative length to zero
neg_to_zero = function(x){return(max(x,0))}
intron_ratio_distance_phyla10_tree$edge.length = sapply(intron_ratio_distance_phyla10_tree$edge.length, neg_to_zero)

In [ ]:
# unroot dated species tree
euk_tree_orig = read.tree(euk_tree_nwk)
euk_tree_orig_phyla10 = keep.tip(euk_tree_orig, intron_ratio_distance_phyla10_tree$tip.label)
euk_tree_phyla10_ur = unroot(euk_tree_orig_phyla10)

In [ ]:
# covert tree edges info into convenient tables
tree_edges_to_df = function(tree){
    tree_edges = as.data.frame(tree$edge)
    colnames(tree_edges) = c('from','to')
    rownames(tree_edges) = 1:nrow(tree_edges)
    tree_edges$length = tree$edge.length
    return(tree_edges)
}

In [ ]:
intron_ratio_distance_phyla10_tree_edges = tree_edges_to_df(intron_ratio_distance_phyla10_tree)
euk_tree_phyla10_ur_edges = tree_edges_to_df(euk_tree_phyla10_ur)

In [ ]:
# match nodes of distance and species tree
node_match = as.data.frame(matchNodes(intron_ratio_distance_phyla10_tree, euk_tree_phyla10_ur))
tip_match = as.data.frame(matchLabels(intron_ratio_distance_phyla10_tree, euk_tree_phyla10_ur))
tree_match = rbind(node_match, tip_match)

In [ ]:
# use node matches and edge tables to match the edges
find_matching_edge_length = function(row){
    from = row[1]
    to = row[2]
    from_match = tree_match[tree_match$tr1 == from,'tr2']
    to_match = tree_match[tree_match$tr1 == to,'tr2']
    if (is.na(from_match) | is.na(to_match)){
        return(0)
    }
    return(euk_tree_phyla10_ur_edges[euk_tree_phyla10_ur_edges$from == from_match & euk_tree_phyla10_ur_edges$to == to_match, 'length'])
}

intron_ratio_distance_phyla10_tree_edges$time = as.numeric(apply(intron_ratio_distance_phyla10_tree_edges, 1, find_matching_edge_length))

In [ ]:
# adjust distance tree branch lengths - divide by the length (time) of the corresponding species tree branch
infnan_to_zero = function(x){
    if (is.nan(x) | is.na(x)){
        return(0)
    }else if (is.infinite(x)){
        return(0)
    }else{
        return(x)
    }
}

In [ ]:
intron_ratio_distance_phyla10_tree$edge.length = sapply(intron_ratio_distance_phyla10_tree_edges$length/(intron_ratio_distance_phyla10_tree_edges$time), infnan_to_zero)

In [ ]:
intron_ratio_distance_phyla10_tree_edges$rate = sapply(intron_ratio_distance_phyla10_tree_edges$length/intron_ratio_distance_phyla10_tree_edges$time, infnan_to_zero)

In [ ]:
# plot tree - color branches by phylum
internal_nodes = unique(intron_ratio_distance_phyla10_tree$edge[,1])
phyla10_edges = list()
for (phy in phy_order){
    phyla10_edges[[phy]] = c()
}

# find clades of same phylum
for (inode in internal_nodes){
    descendant_nodes <- getDescendants(intron_ratio_distance_phyla10_tree, inode)
    tip_labels <- intron_ratio_distance_phyla10_tree$tip.label[descendant_nodes[descendant_nodes <= Ntip(intron_ratio_distance_phyla10_tree)]]
    inode_phyla = unique(gene_structure_stats_df[gene_structure_stats_df$species %in% tip_labels,'phylum'])
    if (length(inode_phyla) == 1){
        edges = which(intron_ratio_distance_phyla10_tree$edge[,2] %in% descendant_nodes)
        phyla10_edges[[inode_phyla]] = unique(c(phyla10_edges[[inode_phyla]], edges))
    }
}

# add edges leading to tips
for (phy in phy_order){
    phy_species = gene_structure_stats_df[gene_structure_stats_df$phylum == phy, 'species']
    phy_species_tip_numbers = which(intron_ratio_distance_phyla10_tree$tip.label %in% phy_species)
    edges = which(intron_ratio_distance_phyla10_tree$edge[,2] %in% phy_species_tip_numbers)
    phyla10_edges[[phy]] = unique(c(phyla10_edges[[phy]], edges))
}

edgecols = c()
edgecols[1:nrow(intron_ratio_distance_phyla10_tree$edge)] = 'black'
i = 1
for (phy in phy_order){
    edgecols[phyla10_edges[[phy]]] = colors[i]
    i = i+1
}

In [ ]:
options(repr.plot.width=60, repr.plot.height=40)
plot(intron_ratio_distance_phyla10_tree, edge.color = edgecols, edge.width=2, show.tip.label = FALSE, type='fan')

In [ ]:
pdf(file='../fig/fig_4b.pdf', width=60, height=60)
options(repr.plot.width=60, repr.plot.height=40)
plot(intron_ratio_distance_phyla10_tree, edge.color = edgecols, edge.width=2, show.tip.label = FALSE, type='fan')
dev.off()

In [ ]:
res = data.frame()
for (phy in phy_order){
    phy_edges = phyla10_edges[[phy]]
    res = rbind(res, colSums(na.omit(intron_ratio_distance_phyla10_tree_edges[phy_edges,c('length','time')])))
}
colnames(res) = c('length','time')
res$phylum = phy_order
res$rate = res$length/res$time*1000

In [ ]:
res

In [ ]:
colSums(na.omit(intron_ratio_distance_phyla10_tree_edges[,c('length','time')]))

__Intron length__

In [ ]:
intron_length_distance_tree = read.tree(intron_length_distance_tree_nwk)
# simlify full species names to binomial names
spec_list = strsplit(intron_length_distance_tree$tip.label, '_')
intron_length_distance_tree$tip.label = unlist(lapply(spec_list, simplify))
# keep only phyla with >=10 sspecies
intron_length_distance_phyla10_tree = keep.tip(intron_length_distance_tree, phyla10_species)
# change branches with negative length to zero
intron_length_distance_phyla10_tree$edge.length = sapply(intron_length_distance_phyla10_tree$edge.length, neg_to_zero)

intron_length_distance_phyla10_tree_edges = tree_edges_to_df(intron_length_distance_phyla10_tree)
euk_tree_phyla10_ur_edges = tree_edges_to_df(euk_tree_phyla10_ur)

# match nodes of distance and species tree
node_match = as.data.frame(matchNodes(intron_length_distance_phyla10_tree, euk_tree_phyla10_ur))
tip_match = as.data.frame(matchLabels(intron_length_distance_phyla10_tree, euk_tree_phyla10_ur))
tree_match = rbind(node_match, tip_match)# match nodes of distance and species tree
node_match = as.data.frame(matchNodes(intron_length_distance_phyla10_tree, euk_tree_phyla10_ur))
tip_match = as.data.frame(matchLabels(intron_length_distance_phyla10_tree, euk_tree_phyla10_ur))
tree_match = rbind(node_match, tip_match)

intron_length_distance_phyla10_tree_edges$time = as.numeric(apply(intron_length_distance_phyla10_tree_edges, 1, find_matching_edge_length))

intron_length_distance_phyla10_tree$edge.length = sapply(intron_length_distance_phyla10_tree_edges$length/(intron_length_distance_phyla10_tree_edges$time), infnan_to_zero)
intron_length_distance_phyla10_tree_edges$rate = sapply(intron_length_distance_phyla10_tree_edges$length/intron_length_distance_phyla10_tree_edges$time, infnan_to_zero)

In [ ]:
# plot tree - color branches by phylum
internal_nodes = unique(intron_length_distance_phyla10_tree$edge[,1])
phyla10_edges = list()
for (phy in phy_order){
    phyla10_edges[[phy]] = c()
}

# find clades of same phylum
for (inode in internal_nodes){
    descendant_nodes <- getDescendants(intron_length_distance_phyla10_tree, inode)
    tip_labels <- intron_length_distance_phyla10_tree$tip.label[descendant_nodes[descendant_nodes <= Ntip(intron_length_distance_phyla10_tree)]]
    inode_phyla = unique(gene_structure_stats_df[gene_structure_stats_df$species %in% tip_labels,'phylum'])
    if (length(inode_phyla) == 1){
        edges = which(intron_length_distance_phyla10_tree$edge[,2] %in% descendant_nodes)
        phyla10_edges[[inode_phyla]] = unique(c(phyla10_edges[[inode_phyla]], edges))
    }
}

# add edges leading to tips
for (phy in phy_order){
    phy_species = gene_structure_stats_df[gene_structure_stats_df$phylum == phy, 'species']
    phy_species_tip_numbers = which(intron_length_distance_phyla10_tree$tip.label %in% phy_species)
    edges = which(intron_length_distance_phyla10_tree$edge[,2] %in% phy_species_tip_numbers)
    phyla10_edges[[phy]] = unique(c(phyla10_edges[[phy]], edges))
}

edgecols = c()
edgecols[1:nrow(intron_length_distance_phyla10_tree$edge)] = 'black'
i = 1
for (phy in phy_order){
    edgecols[phyla10_edges[[phy]]] = colors[i]
    i = i+1
}

In [ ]:
options(repr.plot.width=60, repr.plot.height=40)
plot(intron_length_distance_phyla10_tree, edge.color = edgecols, edge.width=2, show.tip.label = FALSE, type='fan')

In [ ]:
pdf(file='../fig/fig_S3b.pdf', width=60, height=60)
options(repr.plot.width=60, repr.plot.height=40)
plot(intron_ratio_distance_phyla10_tree, edge.color = edgecols, edge.width=2, show.tip.label = FALSE, type='fan')
dev.off()

__Intron count__

In [ ]:
intron_count_distance_tree = read.tree(intron_count_distance_tree_nwk)
# simlify full species names to binomial names
spec_list = strsplit(intron_count_distance_tree$tip.label, '_')
intron_count_distance_tree$tip.label = unlist(lapply(spec_list, simplify))
# keep only phyla with >=10 sspecies
intron_count_distance_phyla10_tree = keep.tip(intron_count_distance_tree, phyla10_species)
# change branches with negative length to zero
intron_count_distance_phyla10_tree$edge.length = sapply(intron_count_distance_phyla10_tree$edge.length, neg_to_zero)

intron_count_distance_phyla10_tree_edges = tree_edges_to_df(intron_count_distance_phyla10_tree)
euk_tree_phyla10_ur_edges = tree_edges_to_df(euk_tree_phyla10_ur)

# match nodes of distance and species tree
node_match = as.data.frame(matchNodes(intron_count_distance_phyla10_tree, euk_tree_phyla10_ur))
tip_match = as.data.frame(matchLabels(intron_count_distance_phyla10_tree, euk_tree_phyla10_ur))
tree_match = rbind(node_match, tip_match)# match nodes of distance and species tree
node_match = as.data.frame(matchNodes(intron_count_distance_phyla10_tree, euk_tree_phyla10_ur))
tip_match = as.data.frame(matchLabels(intron_count_distance_phyla10_tree, euk_tree_phyla10_ur))
tree_match = rbind(node_match, tip_match)

intron_count_distance_phyla10_tree_edges$time = as.numeric(apply(intron_count_distance_phyla10_tree_edges, 1, find_matching_edge_length))

intron_count_distance_phyla10_tree$edge.length = sapply(intron_count_distance_phyla10_tree_edges$length/(intron_count_distance_phyla10_tree_edges$time), infnan_to_zero)
intron_count_distance_phyla10_tree_edges$rate = sapply(intron_count_distance_phyla10_tree_edges$length/intron_count_distance_phyla10_tree_edges$time, infnan_to_zero)

In [ ]:
# plot tree - color branches by phylum
internal_nodes = unique(intron_count_distance_phyla10_tree$edge[,1])
phyla10_edges = list()
for (phy in phy_order){
    phyla10_edges[[phy]] = c()
}

# find clades of same phylum
for (inode in internal_nodes){
    descendant_nodes <- getDescendants(intron_count_distance_phyla10_tree, inode)
    tip_labels <- intron_count_distance_phyla10_tree$tip.label[descendant_nodes[descendant_nodes <= Ntip(intron_count_distance_phyla10_tree)]]
    inode_phyla = unique(gene_structure_stats_df[gene_structure_stats_df$species %in% tip_labels,'phylum'])
    if (length(inode_phyla) == 1){
        edges = which(intron_count_distance_phyla10_tree$edge[,2] %in% descendant_nodes)
        phyla10_edges[[inode_phyla]] = unique(c(phyla10_edges[[inode_phyla]], edges))
    }
}

# add edges leading to tips
for (phy in phy_order){
    phy_species = gene_structure_stats_df[gene_structure_stats_df$phylum == phy, 'species']
    phy_species_tip_numbers = which(intron_count_distance_phyla10_tree$tip.label %in% phy_species)
    edges = which(intron_count_distance_phyla10_tree$edge[,2] %in% phy_species_tip_numbers)
    phyla10_edges[[phy]] = unique(c(phyla10_edges[[phy]], edges))
}

edgecols = c()
edgecols[1:nrow(intron_count_distance_phyla10_tree$edge)] = 'black'
i = 1
for (phy in phy_order){
    edgecols[phyla10_edges[[phy]]] = colors[i]
    i = i+1
}

In [ ]:
options(repr.plot.width=60, repr.plot.height=40)
plot(intron_count_distance_phyla10_tree, edge.color = edgecols, edge.width=2, show.tip.label = FALSE, type='fan')

In [ ]:
pdf(file='../fig/fig_S4b.pdf', width=60, height=60)
options(repr.plot.width=60, repr.plot.height=40)
plot(intron_ratio_distance_phyla10_tree, edge.color = edgecols, edge.width=2, show.tip.label = FALSE, type='fan')
dev.off()

## Intron length - genome size association

In [ ]:
# complete missing genome sizes
l = list()
l$arabidopsis_thaliana = 119667750
l$danaus_plexippus = 248676414
l$fraxinus_excelsior = 867455155
l$onthophagus_taurus = 267079363
l$parasteatoda_tepidariorum = 1228972128
l$quercus_suber = 953298670
l$aegilops_tauschii = 4224915394
l$arabis_alpina = 308032609
l$trichinella_spiralis = 63525422

for (x in names(l)){
    gene_structure_stats_df[gene_structure_stats_df$species == x,'Genome_size'] = l[[x]]
}

In [ ]:
gene_structure_stats_df$log_Genome_size = log10(gene_structure_stats_df$Genome_size)

In [ ]:
rownames(gene_structure_stats_df) = gene_structure_stats_df$species

#### PGLS of all species

In [ ]:
# intron ratio
pgls = PGLS_fossil(modform= Mean_intron_ratio_log ~ log_Genome_size, data=gene_structure_stats_df, RR=intron_ratio_res$rrphylo)
summary(pgls)

In [ ]:
# intron count
pgls = PGLS_fossil(modform= Mean_per_transcript ~ log_Genome_size, data=gene_structure_stats_df, RR=mean_intron_count_res$rrphylo)
summary(pgls)

In [ ]:
# intron length
pgls = PGLS_fossil(modform= log_Mean ~ log_Genome_size, data=gene_structure_stats_df, RR=mean_intron_length_res$rrphylo)
summary(pgls)

#### PGLS per phylum

In [ ]:
features = c('log_Mean', 'Mean_per_transcript', 'Mean_intron_ratio_log')
res = data.frame()
for (phyl in phyla10){
    phyl_species = gene_structure_stats_df[gene_structure_stats_df$phylum == phyl,]$species
    phyl_tree = keep.tip(euk_tree, phyl_species)
    phyl_data = gene_structure_stats_df[gene_structure_stats_df$species %in% phyl_species,]
    phyl_res = c()
    for (feature in features){
        feature_values = phyl_data[[feature]]
        names(feature_values) = phyl_data$species
        feature_rrphylo = RRphylo(tree = phyl_tree, y=feature_values)
        formula = as.formula(paste(feature, '~ log_Genome_size'))
        pgls = PGLS_fossil(modform=formula, data=phyl_data, RR=feature_rrphylo)
        pgls_sum = summary(pgls)
        p_slope = pgls_sum$coefficients[2,4]
        slope = pgls_sum$coefficients[2,1]
        p_intercept = pgls_sum$coefficients[1,4]
        intercept = pgls_sum$coefficients[1,1]
        r2 = pgls_sum$adj.r.squared
        feature_model = c(r2, p_slope, slope, p_intercept, intercept)
        phyl_res = c(phyl_res, feature_model)
    }
    res = rbind(res, phyl_res)
}

In [ ]:
rownames(res) = phyla10
headers = rep(c('R^2','p slope','slope', 'p intercept', 'intercept'),length(features))
headers = paste(headers, rep(features, each=length(headers)/length(features)))
colnames(res) = headers

In [ ]:
res

In [ ]:
write.table(res, sep='\t', file='genome_size_intron_length_PGLS.tsv')